In [ ]:
# poišče graf z najmanjšim ali največjim številom podpoti
# iz random grafa z podanim številom vozlišč in ciklomatičnim št.
#z optimizirano funkcijo za subpath_number
import random as pyrandom
from sage.all import *


# ============================================================
#   HITRA VERZIJA SUBPATH_NUMBER (KLJUČNA OPTIMIZACIJA)
# ============================================================

def subpath_number(G):
    """
    Optimizirana verzija:
    - Za vsak start vertex s naredi en DFS.
    - V enem DFS-u prešteje vse enostavne poti s → v za VSE v.
    - Ne shranjuje poti, samo inkrementira števce.
    - Eksponentna eksplozija je drastično manjša.
    """

    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1     # registriramo pot s → u

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total



# ============================================================
#   Mutacija grafa
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H



# ============================================================
#   Naključni povezan graf G(n, m)
# ============================================================

def random_connected_graph(n, m):
    while True:
        G = graphs.RandomGNM(n, m)
        if G.is_connected():
            return G



# ============================================================
#   Preverjanje PTC
# ============================================================

def is_ptc(G):
    if not G.is_connected():
        return False

    blocks, cut_vertices = G.blocks_and_cut_vertices()
    nontrivial_blocks = [Bl for Bl in blocks if len(Bl) >= 2]
    B = len(nontrivial_blocks)

    if B < 2:
        return False

    triangular_idx = []
    for i, Bl in enumerate(nontrivial_blocks):
        sub = G.subgraph(Bl)
        if sub.order() == 3 and sub.size() == 3:
            triangular_idx.append(i)
        else:
            return False

    k = len(triangular_idx)
    if k < 2:
        return False

    adj = {i: [] for i in triangular_idx}

    for i in triangular_idx:
        for j in triangular_idx:
            if j <= i:
                continue
            inter = set(nontrivial_blocks[i]).intersection(nontrivial_blocks[j])
            if len(inter) > 1:
                return False
            if len(inter) == 1:
                adj[i].append(j)
                adj[j].append(i)

    degrees = [len(adj[i]) for i in triangular_idx]

    if any(d == 0 or d > 2 for d in degrees):
        return False

    ends = sum(1 for d in degrees if d == 1)
    mids = sum(1 for d in degrees if d == 2)

    if ends != 2:
        return False
    if ends + mids != k:
        return False

    start = triangular_idx[0]
    visited = set([start])
    stack = [start]

    while stack:
        v = stack.pop()
        for w in adj[v]:
            if w not in visited:
                visited.add(w)
                stack.append(w)

    return len(visited) == k



# ============================================================
#   Simulated annealing + STOP WHEN PTC FOUND
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3, T_end=0.001,
                        cooling=0.99, max_steps=1000):

    G = random_connected_graph(n, m)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):

        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta / T):
                current_G = new_G
                current_score = new_score

        improved = False

        if direction == "min":
            if current_score < best_score:
                improved = True
        else:
            if current_score > best_score:
                improved = True

        if improved:
            best_score = current_score
            best_G = current_G.copy()

            print(f"Improved at step {step}: best = {best_score}")
            best_G.show()

            if direction == "min" and is_ptc(best_G):
                print("PTC FOUND → stopping SA early!")
                return best_G, best_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score



# ============================================================
#   TEST
# ============================================================

n = 9
c = 20
m = c + n - 1
direction = "min"

best_graph, best_value = simulated_annealing(n, m, direction)
print("Best:", best_value)
best_graph.show()


In [ ]:
# tu je ista funkcija kot prejšnja, le da je še narejena zanka po intervalu št povezav
import random as pyrandom
from sage.all import *

# ============================================================
#   Helper: all simple paths + subpath number
# ============================================================

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths

def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total


# ============================================================
#   Mutacija grafa
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#   Naključni povezan graf G(n, m)
# ============================================================

def random_connected_graph(n, m):
    while True:
        G = graphs.RandomGNM(n, m)
        if G.is_connected():
            return G


# ============================================================
#   Preverjanje PTC
# ============================================================

def is_ptc(G):
    if not G.is_connected():
        return False

    blocks, cut_vertices = G.blocks_and_cut_vertices()
    nontrivial = [Bl for Bl in blocks if len(Bl) >= 2]

    if len(nontrivial) < 2:
        return False

    # vsi nenetrivialni bloki morajo biti trikotniki
    for Bl in nontrivial:
        sub = G.subgraph(Bl)
        if not (sub.order() == 3 and sub.size() == 3):
            return False

    idx = list(range(len(nontrivial)))
    adj = {i: [] for i in idx}

    # povezave med trikotniki
    for i in idx:
        for j in idx:
            if j <= i:
                continue
            inter = set(nontrivial[i]).intersection(nontrivial[j])
            if len(inter) > 1:
                return False
            if len(inter) == 1:
                adj[i].append(j)
                adj[j].append(i)

    deg = [len(adj[i]) for i in idx]

    if any(d == 0 or d > 2 for d in deg):
        return False

    ends = sum(1 for d in deg if d == 1)
    mids = sum(1 for d in deg if d == 2)

    if ends != 2:
        return False
    if ends + mids != len(idx):
        return False

    # preveri povezavnost verige trikotnikov
    start = idx[0]
    visited = {start}
    stack = [start]

    while stack:
        v = stack.pop()
        for w in adj[v]:
            if w not in visited:
                visited.add(w)
                stack.append(w)

    return len(visited) == len(idx)


# ============================================================
#   Simulated annealing (BREZ izrisovanja vmesnih izboljšav)
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=5.0, T_end=0.01,
                        cooling=0.999, max_steps=2000):

    G = random_connected_graph(n, m)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):

        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G, current_score = new_G, new_score
        else:
            if pyrandom.random() < exp(-delta / T):
                current_G, current_score = new_G, new_score

        # preveri izboljšavo
        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_score = current_score
            best_G = current_G.copy()

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#   ZANKA PO INTERVALU m
# ============================================================

n = 8
m_start = 11
m_end   = 13
direction = "min"   # ali "max"

results = []

for m in range(m_start, m_end + 1):
    print(f"\n=== n={n}, m={m} ===")
    best_graph, best_value = simulated_annealing(n, m, direction)
    print(f"Optimalni p_n(G) = {best_value}")
    results.append((m, best_value, best_graph))


# ------------------------------------------------------------
#   Če želiš izrisati samo končne grafe:
# ------------------------------------------------------------

for m, val, G in results:
    print(f"\nKončni graf za m={m}, p_n(G)={val}")
    G.show()


funkcija, ki izriše graf ko podaš ime

In [ ]:
from sage.all import *

g6 = "G~~~~{"   # tukaj daš svoj graph6 zapis
G = Graph(g6)  # iz enoličnega zapisa narediš graf

G.show()       # odpre okno / zriše graf
# ali:
plot(G)        # vrne sliko grafa kot objekt

Poskusam zdej narediti pravo in koncno verzijo

In [ ]:
import random as pyrandom
from sage.all import *
import pandas as pd
import time



def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths


def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total



def save_to_csv(n, mu, graph, score, direction):
    filename = "rezultati_subpath_grafi_enolicni.csv"

    g6 = graph.graph6_string()
    m = graph.size()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n", "µ(G)", "m(G)", "subpath_value",
            "graph6_min", "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # Pretvori tipe
    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    new_row = {
        "n": n_int,
        "µ(G)": mu_int,
        "m(G)": m,
        "subpath_value": score,
        "graph6_min": g6 if direction == "min" else "",
        "graph6_max": g6 if direction == "max" else ""
    }

    # Poišči indeks — tukaj se izognemo vsaki interakciji s SAGE
    matching_indices = df.index[(df["n"] == n_int) & (df["µ(G)"] == mu_int)]

    if len(matching_indices) > 0:
        # matching_indices je Panda Index → PRAVI Python int dobiš tako:
        idx = int(matching_indices.tolist()[0])
        for col, val in new_row.items():
            df.at[idx, col] = val
    else:
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    df.to_csv(filename, index=False)


def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    if len(verts) == 0:
        raise ValueError("Graf nima vozlišč — graph6 zapis je pokvarjen ali prazen.")

    new_v = max(verts) + 1
    H.add_vertex(new_v)

    # STOPNJE KOT SLOVAR {vozlišče: stopnja}
    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if len(degrees) == 0:
        raise ValueError("Graf je imel samo eno vozlišče — ne morem izbrati anchor.")

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    if anchor == new_v:
        raise ValueError(f"Anchor ({anchor}) = new_v ({new_v}) — nekaj je narobe v grafu.")

    H.add_edge(anchor, new_v)
    return H



#še neveva če dela
def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    # če ni nobenega non-edge
    raise ValueError("Graf je že poln — ni več možno dodati ene povezave.")

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("rezultati_subpath_grafi_enolicni.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



def simulated_annealing(n, m, direction,
                        T_start=10.0, T_end=0.001,
                        cooling=0.9995, max_steps=2000,
                        initial_graph=None):

    if initial_graph is not None:
        G = Graph(initial_graph)
    else:
        # fallback = stara logika
        mu = m - n + 1
        if mu >= 1:
            G_old = load_optimal_graph_flexible(n_old=n-1,
                                                mu_desired=mu+1,
                                                direction=direction)
        else:
            G_old = load_optimal_graph_flexible(n_old=n-1,
                                                mu_desired=1,
                                                direction=direction)

        G = expand_graph(G_old, n_new=n, m_target=m, direction=direction)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = ((direction == "min" and current_score < best_score) or
                    (direction == "max" and current_score > best_score))

        if improved:
            best_G = current_G.copy()
            best_score = current_score
            t0 = time.time()
            val = subpath_number(G) 
            print("Subpath time:", time.time() - t0) 

        T *= cooling
        if T < T_end:
            break
        
        


    return best_G, best_score

def compute_all_for_n9(direction):

    n = 9
    mu_max_prev = 21   # za n=8
    mu_max_n9   = 28   # za n=9

    for mu in range(0, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        if mu <= mu_max_prev:
            # 1. DEL — gradimo iz n=8
            G8 = load_G(8, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            # 2. DEL — gradimo iz prejšnjega optimuma n=9
            Gprev = load_G(9, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        # Pretvori µ → m
        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n, 
            m=m, 
            direction=direction,
            initial_graph=Gstart
        )

        print(f"\n=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        best_graph.show()
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)
        
        

# vse tvoje funkcije nad tem …
# …
# …

# === GLAVNI KLIC ===
compute_all_for_n9(direction="min")


problem: ne zuapisuje rezulatet pravilno v csv, naredi stolpce m(G) in subpath value, ter kar nekej vpisuje v stpec max 

In [ ]:
import random as pyrandom
from sage.all import *
import pandas as pd
import time




def subpath_number(G):
    """
    Optimizirana verzija:
    - Za vsak start vertex s naredi en DFS.
    - V enem DFS-u prešteje vse enostavne poti s → v za VSE v.
    - Ne shranjuje poti, samo inkrementira števce.
    - Eksponentna eksplozija je drastično manjša.
    """

    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1     # registriramo pot s → u

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total



def save_to_csv(n, mu, graph, score, direction):
    filename = "rezultati_poskus_n_9_test2.csv"

    g6 = graph.graph6_string()
    m = graph.size()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n", "µ(G)", "m(G)", "subpath_value",
            "graph6_min", "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # Pretvori tipe
    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    new_row = {
        "n": n_int,
        "µ(G)": mu_int,
        "m(G)": m,
        "subpath_value": score,
        "graph6_min": g6 if direction == "min" else "",
        "graph6_max": g6 if direction == "max" else ""
    }

    # Poišči indeks — tukaj se izognemo vsaki interakciji s SAGE
    matching_indices = df.index[(df["n"] == n_int) & (df["µ(G)"] == mu_int)]

    if len(matching_indices) > 0:
        # matching_indices je Panda Index → PRAVI Python int dobiš tako:
        idx = int(matching_indices.tolist()[0])
        for col, val in new_row.items():
            df.at[idx, col] = val
    else:
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    df.to_csv(filename, index=False)


def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    if len(verts) == 0:
        raise ValueError("Graf nima vozlišč — graph6 zapis je pokvarjen ali prazen.")

    new_v = max(verts) + 1
    H.add_vertex(new_v)

    # STOPNJE KOT SLOVAR {vozlišče: stopnja}
    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if len(degrees) == 0:
        raise ValueError("Graf je imel samo eno vozlišče — ne morem izbrati anchor.")

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    if anchor == new_v:
        raise ValueError(f"Anchor ({anchor}) = new_v ({new_v}) — nekaj je narobe v grafu.")

    H.add_edge(anchor, new_v)
    return H



#še neveva če dela
def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    # če ni nobenega non-edge
    raise ValueError("Graf je že poln — ni več možno dodati ene povezave.")

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("rezultati_poskus_n_9_test2.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



def simulated_annealing(n, m, direction,
                        T_start=3.0, T_end=0.001,
                        cooling=0.99, max_steps=300,
                        initial_graph=None):

    if initial_graph is not None:
        G = Graph(initial_graph)
    else:
        # fallback = stara logika
        mu = m - n + 1
        if mu >= 1:
            G_old = load_optimal_graph_flexible(n_old=n-1,
                                                mu_desired=mu+1,
                                                direction=direction)
        else:
            G_old = load_optimal_graph_flexible(n_old=n-1,
                                                mu_desired=1,
                                                direction=direction)

        G = expand_graph(G_old, n_new=n, m_target=m, direction=direction)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = ((direction == "min" and current_score < best_score) or
                    (direction == "max" and current_score > best_score))

        if improved:
            best_G = current_G.copy()
            best_score = current_score
            t0 = time.time()
            val = subpath_number(G) 
            print("Subpath time:", time.time() - t0) 

        T *= cooling
        if T < T_end:
            break
        
        


    return best_G, best_score

def compute_all_for_n9(direction):

    n = 9
    mu_max_prev = 21   # za n=8
    mu_max_n9   = 28   # za n=9

    for mu in range(0, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        if mu <= mu_max_prev:
            # 1. DEL — gradimo iz n=8
            G8 = load_G(8, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            # 2. DEL — gradimo iz prejšnjega optimuma n=9
            Gprev = load_G(9, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        # Pretvori µ → m
        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n, 
            m=m, 
            direction=direction,
            initial_graph=Gstart
        )

        print(f"\n=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        best_graph.show()
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)
        
        

# vse tvoje funkcije nad tem …
# …
# …

# === GLAVNI KLIC ===
compute_all_for_n9(direction="min")


za spodnjo kdo: to je verzija, kjer pravilno zapisuje rezulatate v csv rezultati_poskus_n_9_PRAVILNO

In [ ]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "rezultati_poskus_n_9_PRAVILNO.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    # preveri ali vrstica obstaja
    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    if not mask.any():
        # nova vrstica
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
        }

        if direction == "min":
            new_row["min p_n(G)"] = score
            new_row["graph6_min"] = g6
            new_row["max p_n(G)"] = ""
            new_row["graph6_max"] = ""
        else:
            new_row["min p_n(G)"] = ""
            new_row["graph6_min"] = ""
            new_row["max p_n(G)"] = score
            new_row["graph6_max"] = g6

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    else:
        # posodobi obstoječo vrstico
        idx = df.index[mask][0]     # <-- KLJUČNO (ne Integer(0))

        if direction == "min":
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)



# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("rezultati_poskus_n_9_PRAVILNO.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3.0, T_end=0.001,
                        cooling=0.99, max_steps=300,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n9(direction):

    n = 9
    mu_max_prev = 21
    mu_max_n9 = 28

    for mu in range(0, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        if mu <= mu_max_prev:
            G8 = load_G(8, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            Gprev = load_G(9, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_n9(direction="min")


Obdelujem: n=9, µ=0
=== Rezultati za n=9, µ=0 ===
Best value: 45
graph6: H???F{@


AssertionError: <class 'numpy.int64'>

Spodnja koda zdaj zapisuje minimum in maximum neodvisno od tega ali vrstice v csv ze obstajajo. zapisuje v rezultati_poskus_n_9_PRAVILNO2.csv

In [20]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "rezultati_poskus_n_9_PRAVILNO2.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # poskrbimo, da stolpci obstajajo (če imaš kak star CSV)
    for col in ["min p_n(G)", "max p_n(G)", "graph6_min", "graph6_max"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    # 1) Če vrstice NI → nova
    if not mask.any():
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
            "min p_n(G)": score if direction == "min" else pd.NA,
            "max p_n(G)": score if direction == "max" else pd.NA,
            "graph6_min": g6 if direction == "min" else pd.NA,
            "graph6_max": g6 if direction == "max" else pd.NA
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False)
        return

    # 2) Če vrstica OBSTAJA → update
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if pd.isna(old) or old == "":
            # še nič ni zapisano → zapišemo
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            # že obstaja številka → zamenjamo samo, če je nova boljša (manjša)
            if score < float(old):
                df.at[idx, "min p_n(G)"] = score
                df.at[idx, "graph6_min"] = g6

    else:  # direction == "max"
        old = df.at[idx, "max p_n(G)"]

        if pd.isna(old) or old == "":
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6
        else:
            if score > float(old):
                df.at[idx, "max p_n(G)"] = score
                df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)



# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("rezultati_poskus_n_9_PRAVILNO2.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3.0, T_end=0.001,
                        cooling=0.99, max_steps=300,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n9(direction):

    n = 9
    mu_max_prev = 21
    mu_max_n9 = 28

    for mu in range(0, mu_max_n9 + 1):

        print(f"Obdelujem: n=9, µ={mu}")

        if mu <= mu_max_prev:
            G8 = load_G(8, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            Gprev = load_G(9, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_n9(direction="max")


Obdelujem: n=9, µ=0
=== Rezultati za n=9, µ=0 ===
Best value: 45
graph6: H???F{@
Obdelujem: n=9, µ=1


/tmp/ipykernel_45927/3721436199.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '45' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score
/tmp/ipykernel_45927/3721436199.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '81' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=1 ===
Best value: 81
graph6: H@d?HF?
Obdelujem: n=9, µ=2
=== Rezultati za n=9, µ=2 ===
Best value: 133
graph6: HBgCHHA
Obdelujem: n=9, µ=3


/tmp/ipykernel_45927/3721436199.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '133' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=3 ===
Best value: 221
graph6: HCo`Qi_
Obdelujem: n=9, µ=4


...

/tmp/ipykernel_45927/3721436199.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '321631' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=27 ===
Best value: 397308
graph6: H~~~n~~
Obdelujem: n=9, µ=28


/tmp/ipykernel_45927/3721436199.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '397308' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score


=== Rezultati za n=9, µ=28 ===
Best value: 493209
graph6: H~~~~~~


/tmp/ipykernel_45927/3721436199.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '493209' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "max p_n(G)"] = score
